<a href="https://colab.research.google.com/github/BhagwatiOracle/Deep-Learning/blob/main/KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [5]:
x = df.drop(columns=['Outcome']).values
y=df['Outcome'].values

In [6]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [8]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [9]:
model=Sequential()
model.add(Dense(32,activation='relu',input_shape=(8,)))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.fit(x_train,y_train,batch_size=32,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.6070 - loss: 0.6671 - val_accuracy: 0.6429 - val_loss: 0.6384
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7075 - loss: 0.6138 - val_accuracy: 0.7143 - val_loss: 0.5941
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7376 - loss: 0.5755 - val_accuracy: 0.7727 - val_loss: 0.5602
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7810 - loss: 0.5448 - val_accuracy: 0.7922 - val_loss: 0.5360
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8005 - loss: 0.5113 - val_accuracy: 0.7922 - val_loss: 0.5160
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7603 - loss: 0.5289 - val_accuracy: 0.7857 - val_loss: 0.5025
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7854 - loss: 0.4925 - val_accuracy: 0.7792 - val_loss: 0.4919
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7919 - loss: 0.4773 - val_accuracy: 0.7857 - 

In [11]:
x_train.shape

(614, 8)

In [12]:
!pip install keras-tuner -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00


In [13]:
import keras_tuner as kt

In [14]:
#hp-->hyperparameter
def build_model(hp):
  model=Sequential()

  model.add(Dense(32,activation='relu',input_shape=(8,)))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model


In [15]:
tunner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [16]:
tunner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 4 Complete [00h 00m 04s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 21s


In [17]:
tunner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [18]:
model=tunner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [19]:
model.fit(x_train,y_train,epochs=100,batch_size=32,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7661 - loss: 0.5318 - val_accuracy: 0.7922 - val_loss: 0.5196
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7596 - loss: 0.5160 - val_accuracy: 0.7922 - val_loss: 0.5069
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7811 - loss: 0.4886 - val_accuracy: 0.7792 - val_loss: 0.4980
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7582 - loss: 0.5015 - val_accuracy: 0.7792 - val_loss: 0.4917
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7954 - loss: 0.4633 - val_accuracy: 0.7792 - val_loss: 0.4867
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7658 - loss: 0.4855 - val_accuracy: 0.7792 - val_loss: 0.4837
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7888 - loss: 0.4732 - val_accuracy: 0.7857 - val_loss: 0.4808
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7951 - loss: 0.4583 - val_accuracy: 0.79

In [20]:
def build_model(hp):
  model=Sequential()

  model.add(Dense(72,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [22]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='num_layers')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 08s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 21s


In [24]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6}

In [25]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [26]:
model.fit(x_train,y_train,epochs=100,initial_epoch=9,validation_data=(x_test,y_test))

Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.7744 - loss: 0.4537 - val_accuracy: 0.7727 - val_loss: 0.4771
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7910 - loss: 0.4304 - val_accuracy: 0.7922 - val_loss: 0.4652
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7944 - loss: 0.4272 - val_accuracy: 0.7922 - val_loss: 0.4717
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8104 - loss: 0.4292 - val_accuracy: 0.7857 - val_loss: 0.4831
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8488 - loss: 0.3712 - val_accuracy: 0.7792 - val_loss: 0.4730
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8075 - loss: 0.4116 - val_accuracy: 0.7922 - val_loss: 0.4777
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8668 - loss: 0.3328 - val_accuracy: 0.7857 - val_loss: 0.4718
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8726 - loss: 0.3296 - val_accura

In [36]:
def build_model(hp):
  model=Sequential()
  counter=0

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter == 0:
      model.add(Dense(
          hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
          activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),
          input_shape=(8,)
      ))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))


    else:
      model.add(Dense(
          hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
          activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])
      ))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
      counter+=1

  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']), loss='binary_crossentropy',metrics=['accuracy'])

  return model



In [37]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='my_dir',project_name='final')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [40]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.649350643157959

Best val_accuracy So Far: 0.649350643157959
Total elapsed time: 00h 00m 17s


In [41]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10,
 'units0': 40,
 'activation0': 'sigmoid',
 'dropout0': 0.9,
 'optimizer': 'sgd',
 'units1': 16,
 'activation1': 'relu',
 'dropout1': 0.7,
 'units2': 72,
 'activation2': 'tanh',
 'dropout2': 0.3,
 'units3': 56,
 'activation3': 'tanh',
 'dropout3': 0.8,
 'units4': 40,
 'activation4': 'tanh',
 'dropout4': 0.2,
 'units5': 24,
 'activation5': 'sigmoid',
 'dropout5': 0.6,
 'units6': 120,
 'activation6': 'relu',
 'dropout6': 0.6,
 'units7': 56,
 'activation7': 'relu',
 'dropout7': 0.1,
 'units8': 128,
 'activation8': 'tanh',
 'dropout8': 0.8,
 'units9': 104,
 'activation9': 'relu',
 'dropout9': 0.5}

In [42]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 1.0000 - loss: 2.1632e-05 - val_accuracy: 0.6558 - val_loss: 3.2957
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 2.4253e-05 - val_accuracy: 0.6558 - val_loss: 3.3141
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 1.9172e-05 - val_accuracy: 0.6558 - val_loss: 3.3445
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 2.3234e-05 - val_accuracy: 0.6558 - val_loss: 3.3643
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 2.0608e-05 - val_accuracy: 0.6558 - val_loss: 3.3884
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 1.8190e-05 - val_accuracy: 0.6558 - val_loss: 3.4077
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 1.5140e-05 - val_accuracy: 0.6558 - val_loss: 3.4217
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: